In [1]:
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.utils.estimator_checks import (
    check_estimator,
)

from sippy_unipi.datasets import gen_gbn_seq, white_noise
from sippy_unipi.io import ARMAX, ARX, BJ, FIR, GEN
from sippy_unipi.model_selection import aic_scorer
from sippy_unipi.ss import (
    CVA,
    MOESP,
    N4SID,
    ParsimK,
    ParsimP,
    ParsimS,
    SSModel,
)

# Enable automatic reloading of modules when they change
%load_ext autoreload
%autoreload 2

In [2]:
# Sample time
ts = 1.0

# SISO SS system (n = 2)
A = np.array([[0.89, 0.0], [0.0, 0.45]])
B = np.array([[0.3], [2.5]])
C = np.array([[0.7, 1.0]])
D = np.array([[0.0]])

sys_true = SSModel._from_state(A, B, C, D)
tfin = 500
npts = int(tfin // ts) + 1
Time = np.linspace(0, tfin, npts)

In [ ]:
# Input sequence
U = np.zeros((1, npts))
U[0] = gen_gbn_seq(npts, 0.05)

# Output
yout = sys_true.predict(U.T)

# Measurement noise
noise = white_noise(0.15, (npts,)).T

# Output with noise
y_tot = yout + noise

U = U.T
U.shape, y_tot.shape

In [ ]:
estimators = [
    ARMAX(),
    ARX(),
    FIR(),
    # ARMA(),
    # OE(),
    BJ(),
    GEN(),
    ParsimK(),
    ParsimP(),
    ParsimS(),
    CVA(),
    N4SID(),
    MOESP(),
]
expected_failed_checks = {
    "check_methods_sample_order_invariance": "The methods are not order invariant",
    "check_methods_subset_invariance": "The methods are not subset invariant",
}
for estimator in estimators:
    try:
        check_estimator(
            estimator,
            expected_failed_checks=expected_failed_checks,
        )
    except Exception as e:
        print(f"Error checking {estimator.__class__.__name__}: {e}")
        raise e


In [ ]:
parsim = ARMAX()
parsim_search = GridSearchCV(
    parsim,
    {"na": [1, 2, 3, 4, 5]},
    scoring=aic_scorer,
)
parsim_search.fit(U.reshape(-1, 1), y_tot.reshape(-1, 1))